# SVN Reports Dashboard
Reporting import/upload pipelines

### Import Libraries
Libraries necessary to parse through each dashboard

In [1]:
from IPython.display import display, Markdown, Latex
import pandas as pd

### Set up Table Styles
Use `pandas.style` to set up some graphical styles for the main reports table.

In [2]:
table_styles = [{
    'selector': 'td',
    'props': [('background-color', '#808080'), ('color', '#FFFFFF')]
}, 
{
    'selector': 'th',
    'props': [('background-color', '#000000'), ('color', '#FF0000')]
}]

### Define Functions
Define functions to parse through each of the sites and print them to the screen

In [3]:
sites = ["Reports"]
durations = [3, 30, 3000]

# In future iterations, display path is:  '../../../../../log/status_reports/'
def load_dataframe_and_display(file_name=None):
    # Header and load in dataframe
    display(Markdown('## SLA ' + file_name + ' Dashboard'))
    df = pd.read_csv('../status_reports/' + file_name.lower() + '.csv')
    
    # Categorize each report by SLA (3, 30, 300 -> Dead)
    for duration in durations:
        duration_df = df.loc[df['sla'] == duration]
        duration_str = str(duration) + '-Day'
        if (duration == 300):
            duration_str = 'Dead'
        display(Markdown(duration_str + ' SLA Laptop Report'))
        display(duration_df.style.set_table_styles(
            table_styles
        ))

def main():
    for site in sites:
        load_dataframe_and_display(site)

### Run all
Run functions and produce dashboards for each site

In [4]:
main()

## SLA Reports Dashboard

3-Day SLA Laptop Report

,laptop,date_updated,time_diff,sla,sla_percentage
21,ucsd51,2021-08-11 18:17:27.507785+00:00,12 days 01:16:04.064477,3,4.017608
71,ucsd49,2021-08-23 07:03:31.587585+00:00,0 days 12:29:57.973886,3,0.173603


30-Day SLA Laptop Report

,laptop,date_updated,time_diff,sla,sla_percentage
0,ucsd22,2018-04-04 22:47:22.688156+00:00,1236 days 20:46:07.785359,30,41.228846
1,duke30,2018-04-10 14:57:58.116245+00:00,1231 days 04:35:31.737347,30,41.039711
2,ohsu20,2018-04-16 16:16:20.761852+00:00,1225 days 03:17:08.966946,30,40.837897
3,duke19,2018-04-17 18:06:57.680815+00:00,1224 days 01:26:34.027264,30,40.802004
4,ohsu21,2018-04-20 15:35:23.994334+00:00,1221 days 03:58:08.639432,30,40.705513
5,duke18,2018-04-28 20:05:31.394105+00:00,1212 days 23:27:59.632612,30,40.432592
6,sri52,2018-09-27 01:15:36.846530+00:00,1061 days 18:17:51.891887,30,35.392080
7,ohsu52,2018-10-05 18:23:32.796429+00:00,1053 days 01:09:55.758362,30,35.101619
8,duke7,2019-03-18 16:19:52.037297+00:00,889 days 03:13:38.560280,30,29.637816
9,duke31,2019-03-18 18:27:38.047836+00:00,889 days 01:05:53.844224,30,29.634859


3000-Day SLA Laptop Report

,laptop,date_updated,time_diff,sla,sla_percentage
23,admin2,2013-05-03 16:45:50.044002+00:00,3034 days 02:47:39.358977,3000,1.011372
24,admin36,2013-08-25 18:25:55.584166+00:00,2920 days 01:07:32.996434,3000,0.973349
25,admin35,2013-08-25 18:26:01.240946+00:00,2920 days 01:07:26.356048,3000,0.973349
26,ohsu-merge,2013-10-31 23:31:06.948094+00:00,2852 days 20:02:24.599735,3000,0.950945
28,admin1,2014-03-20 17:39:25.734019+00:00,2713 days 01:54:06.720063,3000,0.904360
29,sri38,2014-03-20 18:56:22.582503+00:00,2713 days 00:37:06.278852,3000,0.904342
30,ucsd-merge,2014-07-24 17:52:07.152474+00:00,2587 days 01:41:25.540620,3000,0.862357
31,sri-merge,2014-07-24 17:52:07.152474+00:00,2587 days 01:41:24.155319,3000,0.862357
32,upmc-merge,2014-07-24 17:52:07.152474+00:00,2587 days 01:41:20.473706,3000,0.862357
33,duke-merge,2015-03-31 23:11:20.513258+00:00,2336 days 20:22:10.190914,3000,0.778950
